In [2]:
import requests
import base64 
import datetime
from urllib.parse import urlencode
import pprint

In [2]:
client_id = 'client id'
client_secret = 'client secret'

In [17]:
class SpotifyAPI():
    client_id = None
    client_secret = None
    access_token = None
    access_token_expires_in = datetime.datetime.now()
    access_token_did_expire = False
    token_url = "https://accounts.spotify.com/api/token"

    def __init__(self, client_id , client_secret):
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None:
            raise Exception("You must set client id and client secret")
        client_creds = f"{self.client_id}:{self.client_secret}"
        client_creds_base64 = base64.b64encode(client_creds.encode())
        return client_creds_base64
    
    def get_token_header(self):
        client_creds_base64 = self.get_client_credentials()
        token_header = {
        "Authorization": f"Basic {client_creds_base64.decode()}"  # <base64 encoded client_id:client_secret>
        }
        return token_header
          
    def get_token_data(self):
        return {
                "grant_type":"client_credentials"
                }

    def perform_auth(self):
        token_data = self.get_token_data()
        token_header = self.get_token_header()
        r = requests.post(self.token_url, data = token_data, headers =token_header)
        if r.status_code not in range(200,299):
#             print(r.status_code)
            return False
        
        data = r.json()
        access_token = data['access_token']
        expires_in = data['expires_in']
        now = datetime.datetime.now()
        expires = now + datetime.timedelta(seconds = expires_in) # to get the time of expiration 
        self.access_token = access_token
        self.access_token_expires_in = expires
        self.access_token_did_expire = expires < now # token expired = if time of expiration is less than current time 
        return True
        
        

In [20]:
spotify = SpotifyAPI(client_id , client_secret)
spotify.perform_auth()

True

In [21]:
token_header = {
   "Authorization" : f"Bearer {spotify.access_token}"
}
endpoint = "https://api.spotify.com/v1/search"
data = urlencode({ "q":"tania", "type": "artist" })
lookup_url = f"{endpoint}{data}"
# print(lookup_url)    --   https://api.spotify.com/v1/searchq=Time&type=track 
# print(data)       --  q=Time&type=track
r = requests.get(f"https://api.spotify.com/v1/search?{data}", headers = token_header)
pprint.pprint(r.json())

{'artists': {'href': 'https://api.spotify.com/v1/search?query=tania&type=artist&offset=0&limit=20',
             'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4rfeqiQmvXlc6LYxsgrXY7'},
                        'followers': {'href': None, 'total': 9},
                        'genres': [],
                        'href': 'https://api.spotify.com/v1/artists/4rfeqiQmvXlc6LYxsgrXY7',
                        'id': '4rfeqiQmvXlc6LYxsgrXY7',
                        'images': [{'height': 640,
                                    'url': 'https://i.scdn.co/image/ab67616d0000b2734b17d92e59bf8b0953397803',
                                    'width': 640},
                                   {'height': 300,
                                    'url': 'https://i.scdn.co/image/ab67616d00001e024b17d92e59bf8b0953397803',
                                    'width': 300},
                                   {'height': 64,
                                    'url': 'https://i.scdn.c

                                    'width': 300},
                                   {'height': 64,
                                    'url': 'https://i.scdn.co/image/ab67616d00004851fdeecd960c75f4b2f9b5920d',
                                    'width': 64}],
                        'name': 'Tania Davis',
                        'popularity': 25,
                        'type': 'artist',
                        'uri': 'spotify:artist:1I2ZfQKQQyFgHpbmNLxqZq'},
                       {'external_urls': {'spotify': 'https://open.spotify.com/artist/6d82VM0QVVmxJoLfQ3KyrI'},
                        'followers': {'href': None, 'total': 217},
                        'genres': [],
                        'href': 'https://api.spotify.com/v1/artists/6d82VM0QVVmxJoLfQ3KyrI',
                        'id': '6d82VM0QVVmxJoLfQ3KyrI',
                        'images': [{'height': 640,
                                    'url': 'https://i.scdn.co/image/0f8b46628d5bfe7a1862753c9ca904372e95cbd1',
     